In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

In [3]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [6]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
    percentage = 1
    data_loader = DataLoader(percentage, True, param['random_seed'])
    train_df, test_df = data_loader.create_train_test_df(True, True, True)
    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    print(best_estimator.get_all_performance_metrics())
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
    }
    print(output)
    results.append(output)

Number of experiments to run:  1
Numerical Rating
1    2990
3     619
Name: count, dtype: int64
Numerical Rating
1    778
3    124
Name: count, dtype: int64
cluster  predict  veracity
-1       False     1          1437
                   3           200
         True     -1           609
 0       False     1             3
         True     -1             2
                              ... 
 66      True     -1             5
 67      False     1             3
         True     -1             5
 68      False     1            13
         True     -1            12
Name: count, Length: 121, dtype: int64
breaking further...
cluster  predict  veracity
-1       False     1          1437
                   3           200
         True     -1           609
 0       False     1             3
         True     -1             2
                              ... 
 66      True     -1             5
 67      False     1             3
         True     -1             5
 68      False     1          

In [7]:
results_df = pd.json_normalize(results)
results_df

,percentage,score,accuracy,was_supervised_umap_used,metrics.accuracy,metrics.accuracy_not_including_fours,metrics.percentage_of_fours,metrics.percentage_of_no_clusters_in_ground_truth,metrics.precision_on_three,metrics.recall_on_three,metrics.average_confidence_for_3,metrics.precision_on_three_excluding_fours,metrics.recall_on_three_excluding_fours,metrics.cluster_df
0,1,1.666667,0.311674,True,0.311674,0.982639,0.682819,0.0,1.0,0.080645,0,1.0,0.666667,...


In [8]:
cluster_dfs[0]

,text,veracity,predict,predicted_veracity,embeddings,cluster,num_correct_in_cluster,total_in_cluster,cluster_accuracy
0,Video shows geoengineering operation blocking ...,1,True,4,"[5.639001369476318, -1.0828005075454712, -7.34...",-1,1637,2246,0.728851
1,EU members states have spent about ?600 billio...,1,True,4,"[5.900769233703613, -1.2067428827285767, -7.90...",-1,1637,2246,0.728851
2,US billionaire George Soros attended the UN’s ...,1,True,4,"[5.6700615882873535, -1.0941205024719238, -7.4...",-1,1637,2246,0.728851
3,"DOCKET 3: Methane Emissions Reduction Program,...",3,True,4,"[5.874303340911865, -1.1946101188659668, -7.76...",-1,1637,2246,0.728851
4,The skepticism is up from an already high 48 p...,1,True,1,"[5.750396251678467, -1.1976326704025269, -7.39...",58,10,10,1.000000
...,...,...,...,...,...,...,...,...,...
3796,All U.S. regions are experiencing the impacts ...,3,False,3,"[5.842842102050781, -1.124656319618225, -7.418...",14,123,123,1.000000
3797,This is roughly equivalent to the greenhouse g...,3,False,3,"[5.977087497711182, -1.2822843790054321, -7.64...",-1,1637,2246,0.728851
3798,"Since 2000, PM2.5 concentrations in the outdoo...",3,False,3,"[5.939883232116699, -1.2525899410247803, -7.60...",-1,1637,2246,0.728851
3799,EPA is currently developing new content that p...,3,False,3,"[5.841381549835205, -1.130470633506775, -7.507...",-1,1637,2246,0.728851


In [ ]:
viz = Visualizer()
df_with_two_dimens = viz.fit_transform(cluster_dfs[0], 'embeddings')
df_with_two_dimens